In [1]:
import sys

sys.path.append('..')

In [2]:
from pathlib import Path

from herbarium.pylib import db
from herbarium.pylib.multi_efficient_net import NETS
from herbarium.pylib.train_model import test

In [3]:
DATA = Path('..') / 'data'
MODELS = DATA / 'models'

In [4]:
class Args:
    net: str = 'b0'
    database: Path = DATA / 'angiosperms.sqlite'
    load_weights: Path = MODELS / 'b0_flowers_2_orders_unfrozen_2_acc.pth'
    split_run: str = 'flowers_2_orders'
    workers: int = 8
    limit: int = 0
    batch_size: int = 32
    trait: str = ['flowering']


args = Args()

In [5]:
orders = db.select_orders(args.database, args.split_run)

net = NETS[args.net](len(orders), args.load_weights, freeze='all')

In [6]:
test(args, net, orders)

2021-12-21 16:52:53 INFO: ================================================================================
2021-12-21 16:52:53 INFO: ipykernel_launcher started
2021-12-21 16:53:45 INFO: Test: loss 0.114918 acc 0.955019
2021-12-21 16:53:45 INFO: ipykernel_launcher finished
